In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from skimage import filters
from skimage import morphology
import math
from skimage import measure
import time
import sklearn.preprocessing
from sklearn.neighbors import KNeighborsClassifier
%load_ext Cython

In [2]:
def img_normalize(sv):
    mx, mn=sv.max(), sv.min()
    sv=(sv-mn)/(mx-mn)
    sv=np.asarray(sv*255, dtype='uint8')
    return sv


dt=[('0' if x<10 else '')+str(x) for x in range(1, 15)]
dy=[[y+'_'+x for y in dt] for x in ['dr', 'h', 'g']]
z=[]
for x in dy:
    z=z+x
print(z)

flag=cv2.IMREAD_GRAYSCALE
#flag=cv2.IMREAD_COLOR
Manuals = [cv2.imread(f"manual1/{x}.tif", flag) for x in z]
Masks= [cv2.imread(f"mask/{x}_mask.tif", flag) for x in z]
Images= [cv2.imread(f"images/{x}.jpg", flag) for x in z]

zupaImagez=[cv2.imread(f"images/{x}.jpg", cv2.IMREAD_COLOR) for x in z]

Creation=list(zip(Manuals, Masks, Images, zupaImagez))

"""for x in Images:
    #plt.imshow(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
    plt.imshow(x, cmap='gray')
    plt.show()
"""

['01_dr', '02_dr', '03_dr', '04_dr', '05_dr', '06_dr', '07_dr', '08_dr', '09_dr', '10_dr', '11_dr', '12_dr', '13_dr', '14_dr', '01_h', '02_h', '03_h', '04_h', '05_h', '06_h', '07_h', '08_h', '09_h', '10_h', '11_h', '12_h', '13_h', '14_h', '01_g', '02_g', '03_g', '04_g', '05_g', '06_g', '07_g', '08_g', '09_g', '10_g', '11_g', '12_g', '13_g', '14_g']


"for x in Images:\n    #plt.imshow(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))\n    plt.imshow(x, cmap='gray')\n    plt.show()\n"

In [3]:
def pad(img, con=3):
    if (len(img.shape)==2):
        dt=np.zeros((img.shape[0]+2*con, img.shape[1]+2*con), dtype='uint8')
        dt[con:dt.shape[0]-con,con:dt.shape[1]-con]=img
    else:
        dt=np.zeros((img.shape[0]+2*con, img.shape[1]+2*con, img.shape[2]), dtype='uint8')
        dt[con:dt.shape[0]-con,con:dt.shape[1]-con]=img
    
    return dt

def getItem(img, x, y, con=3, img2=None):
    #huhu=measure.moments_hu(img[x-con:x+con,y-con:y+con])
    if (img2 is None):
        mcen=measure.moments_central(img[x-con:x+con,y-con:y+con])
        vz=[img[x-con:x+con,y-con:y+con].var()]
    else:
        mcen=measure.moments_central(img2[x-con:x+con,y-con:y+con])
        vz=[img2[x-con:x+con,y-con:y+con,z].var() for z in range(3)]
    col=[img[x,y]]
    lees=np.concatenate((vz, col, np.reshape(mcen, mcen.shape[0]*mcen.shape[1]*mcen.shape[2])))
    return lees

def getImagoData(img, imgalc, mask):
    Cnt=5
    #dissolve=np.zeros((img.shape[0]*img.shape[1], getItem(img, 200, 200).shape[0]))
    dissolve=[]
    img2=pad(img, Cnt)
    img3=pad(imgalc, Cnt)
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            #dissolve[i*img.shape[0]+j]=getItem(img2, i+Cnt, j+Cnt, Cnt, img3)
            if (mask[i,j]!=0):
                dissolve.append(getItem(img2, i+Cnt, j+Cnt, Cnt, img3))
    return dissolve


def getImagoRes(bwimg):
    res=np.reshape(bwimg, (bwimg.shape[0]*bwimg.shape[1]))
    return res

def getImagoRandomoRes(resimg, tupls):
    res=[]
    for x in tupls:
        res.append(resimg[x[0], x[1]])
    return np.asarray(res)

#0 - manual, 1 - maska, 2 - oryginał
def randomizer(set_img, Z=40001):
    Cnt=5
    img2=pad(set_img[2], Cnt)
    img3=pad(set_img[3], Cnt)
    dissolve=[]
    tuplez=[]
    setall=set()
    
    for i in range(Z):
        while (True):
            rn=np.random.randint(0, set_img[0].shape[0]*set_img[0].shape[1])
            rny=rn%set_img[1].shape[1]
            rnx=rn//set_img[1].shape[1]
                
            if (set_img[1][rnx,rny]!=0 and
                (set_img[0][rnx,rny]!=0 or
                 np.random.randint(0, 4)%4==0) and
               {(rnx, rny)} not in setall):
                break
        dissolve.append(getItem(img2, rnx+Cnt, rny+Cnt, Cnt, img3))
        setall|={(rnx, rny)}
        tuplez.append((rnx, rny))
        #print(rnx, rny, set_img[0][rnx,rny])
    return np.asarray(dissolve), tuplez

def preproc(data):
    dv=sklearn.preprocessing.normalize(data)
    return dv

In [4]:
t1=time.time()
x1, locs1=randomizer(Creation[0])
y1=getImagoRandomoRes(Creation[0][0], locs1)

##Normalizacja trainseta, uwypuklenie kluczowych cech
xp1=preproc(x1)

ncl=KNeighborsClassifier(n_neighbors=5)
ncl.fit(xp1, y1)
t2=time.time()
print(t2-t1)

8.889613628387451


In [5]:
def evaluator(im1, im2):
    dicc={}
    dicc["1st B"]=len(im1[im1==0])
    dicc["1st W"]=len(im1[im1!=0])
    dicc["2nd B"]=len(im2[im2==0])
    dicc["2nd W"]=len(im2[im2!=0])
    
    dicc["tp"]=len(im2[(im2==im1) & (im2!=0)])
    dicc["tn"]=len(im2[(im2==im1) & (im2==0)])
    dicc["fp"]=len(im2[(im2!=im1) & (im2!=0)])
    dicc["fn"]=len(im2[(im2!=im1) & (im2==0)])
    _=[print(x, dicc[x]) for x in ["tp", "tn", "fp", "fn"]]
    if (dicc["fn"]==0):
        dicc["fn"]=1
    
    dicc["acc"]=(dicc["tp"]+dicc["tn"])/(dicc["tp"]+dicc["tn"]+dicc["fp"]+dicc["fn"])
    dicc["prec"]=dicc["tp"]/(dicc["tp"]+dicc["fp"])
    dicc["rec"]=dicc["tp"]/(dicc["tp"]+dicc["fn"])
    
    return dicc

for i in range(1, 3):
    t1=time.time()
    
    xt, locst=randomizer(Creation[i])
    yt=getImagoRandomoRes(Creation[i][0], locst)
    xpt=preproc(xt)
    ypt=ncl.predict(xpt)
    dct=evaluator(yt, ypt)
    print(f"Obraz {i+1}:")
    print(f"acc:{dct['acc']}, prec:{dct['prec']}, rec:{dct['rec']}, geom:{math.sqrt(dct['prec']*dct['rec'])}")
    
    t2=time.time()
    print(t2-t1)

tp 4402
tn 29001
fp 1178
fn 5420
Obraz 2:
acc:0.8350541236469088, prec:0.7888888888888889, rec:0.4481775605782936, geom:0.5946110474835989
37.14584422111511
tp 2936
tn 29573
fp 875
fn 6617
Obraz 3:
acc:0.8127046823829405, prec:0.7704014694305956, rec:0.3073380090024076, geom:0.48659393106298443
49.81818699836731


In [27]:
c0=1000
c1=600
Im=1
dissolve=[]
Cnt=5

t1=time.time()
img2=pad(Creation[Im][2], Cnt)
img3=pad(Creation[Im][3], Cnt)
for i in range(c0, c0+c1):
    for j in range(c0, c0+c1):
        dissolve.append(getItem(img2, i+Cnt, j+Cnt, Cnt, img3))

t2=time.time()
print(t2-t1)

64.52344965934753


In [ ]:
t1=time.time()
dt=np.zeros((c1, c1), dtype='float64')
xv=preproc(dissolve)
ypt=ncl.predict(np.asarray(xv))

ij=0
for i in range(c0, c0+c1):
    for j in range(c0, c0+c1):
        dt[i-c0,j-c0]=ypt[ij]
        ij+=1

t2=time.time()
print(t2-t1)

In [ ]:
fig, ax=plt.subplots(1, 2, figsize=(18,18))
ax[1].imshow(Creation[Im][0][c0:c0+c1,c0:c0+c1], cmap='gray')
ax[0].imshow(dt, cmap='gray')
plt.show() 

In [287]:
print(Creation[0][0][Creation[0][0]>0])

[255 255 255 ... 255 255 255]


In [195]:
print(len(Creation[0][2][Creation[0][2]==0]))
print(len(Creation[0][2][Creation[0][2]>0]))
print(Creation[0][2].max())

871079
7314265
186


In [31]:
np.asarray([[1,2,3],[4,5,6]]).var()

2.9166666666666665

In [ ]:
%%cython
import numpy as np
cimport numpy as np
from cpython cimport array
from skimage import measure
import array


def pad(img, con=3):
    dt=np.zeros((img.shape[0]+2*con, img.shape[1]+2*con), dtype='uint8')
    dt[con:dt.shape[0]-con,con:dt.shape[1]-con]=img
    return dt


def getItem(img, x, y, con=3):
    lees=np.asarray([], dtype='float')
    huhu=measure.moments_hu(img[x-con:x+con,y-con:y+con])
    lees=np.concatenate((lees, huhu))
    col=img[x,y]
    lees=np.concatenate((lees, np.asarray([col], dtype='float')))
    return lees

def getImagoData(unsigned char[:,:] img, int train=0):
    #cdef int s=0, t=0, gr=0, ij, i, jj
    cdef int i=0, j=0, Cnt=10, is1=img.shape[0], is2=img.shape[1], ij
    cdef np.ndarray[double, ndim=2] dissolve=np.zeros((img.shape[0]*img.shape[1], getItem(img, 200, 200).shape[0]))
    cdef np.ndarray[unsigned char, ndim=2] img2=pad(img, Cnt)
    cdef np.ndarray[double, ndim=1] tmp=np.zeros((getItem(img, 200, 200).shape[0]))
    
    for i in range(is1):
        for j in range(is2):
            tmp=getItem(img2, i+Cnt, j+Cnt, Cnt)
            for ij in range(tmp.shape[0]):
                dissolve[i*is1+j,ij]=tmp[ij]
    return dissolve

In [46]:
res=np.asarray([[1,2,3,4],[5,6,7,8]])
print(np.reshape(res, (res.shape[0]*res.shape[1])))



[1 2 3 4 5 6 7 8]
-3364981360.0
